In [1]:
import sys
sys.path.append("../")

In [2]:
%load_ext autoreload
%autoreload 2
from ml_hep_tracking.models import DeepNeuralNet, LGBM
from ml_hep_tracking.preprocessing import data_pipeline
from ml_hep_tracking.utils import get_data, shuffle_arrays

Using TensorFlow backend.


In [3]:
import pandas as pd

In [4]:
import pathlib
DATASET_PATH = pathlib.Path('../data_mlhep2017/DS_1_train.csv')

train_df, test_df = get_data(DATASET_PATH)

In [5]:
train_df.head()

,index,event_id,X,Y,Z,TX,TY,chi2,signal
478529,478529,-999,70536.734375,54893.789062,63357.0,0.399330,0.124349,1.893766,0
906933,906933,-999,51661.796875,66668.953125,34911.0,-0.059654,0.403125,0.384596,0
771028,771028,-999,71366.585938,66274.781250,40083.0,-0.286347,-0.380841,2.582410,0
326743,326743,-999,41645.906250,12144.453125,60771.0,0.161012,0.120340,2.783556,0
891041,891041,-999,39456.769531,43884.652344,54306.0,0.297656,-0.205432,2.553855,0


In [6]:
train_df.describe()

,index,event_id,X,Y,Z,TX,TY,chi2,signal
count,7.862230e+05,786223.000000,786223.000000,786223.000000,786223.000000,786223.000000,786223.000000,786223.000000,786223.000000
mean,5.613046e+05,-51.735681,50187.278986,40191.555223,36948.293627,-0.000025,0.001162,1.973560,0.045516
std,3.241373e+05,5436.912217,12444.820593,20532.655764,21281.756755,0.302364,0.317821,0.760289,0.208434
min,0.000000e+00,-999.000000,27762.531250,3319.980469,0.000000,-0.996548,-0.999044,0.019624,0.000000
25%,2.805800e+05,-999.000000,39504.382812,22219.257812,18102.000000,-0.233663,-0.250502,1.425806,0.000000
50%,5.608230e+05,-999.000000,50380.003906,40276.746094,37497.000000,-0.000260,0.003078,2.150352,0.000000
75%,8.420020e+05,-999.000000,60793.224609,57975.843750,55599.000000,0.235007,0.251395,2.614520,0.000000
max,1.123176e+06,49513.000000,72227.281250,76694.054688,72408.000000,0.998503,0.999602,3.000000,1.000000


In [7]:
len(train_df['event_id'].unique())

101

In [8]:
train_df['event_id'].value_counts()

-999      750437
 411        2107
 24423      1527
 6654       1489
 12128      1263
 9110        903
 5294        807
 739         768
 14007       766
 33945       765
 5586        747
 43358       736
 45475       732
 27558       674
 912         667
 1919        661
 32733       613
 27354       591
 6551        589
 38637       579
 11904       554
 49513       529
 7027        496
 3009        467
 3244        452
 17847       428
 36172       425
 46418       419
 38383       410
 5207        376
           ...  
 22396       172
 48293       169
 29          168
 710         165
 28229       161
 14451       157
 11548       156
 36064       155
 43048       150
 9674        150
 3731        144
 35192       141
 36057       137
 30167       130
 6780        128
 7716        125
 38720       123
 32725       121
 26280       114
 30603       108
 36628       108
 18740       104
 19167       104
 7069         98
 29180        96
 22207        94
 14155        93
 10180        

In [9]:
true_events = train_df[train_df['event_id'] != -999].copy()

In [10]:
true_events.head()

,index,event_id,X,Y,Z,TX,TY,chi2,signal
173027,173027,5294,54542.996094,23498.103516,59478.0,-0.267959,0.261389,0.668103,1
483923,483923,3009,54336.457031,49987.535156,46548.0,0.070085,0.025079,0.390442,1
794522,794522,5586,44833.660156,57492.250000,47841.0,-0.125414,0.163239,0.360280,1
310978,310978,29,40527.925781,28469.480469,63357.0,0.363918,-0.078953,1.989602,1
805462,805462,5294,58759.496094,20945.890625,58185.0,-0.081031,0.061928,0.708806,1


In [11]:
events = true_events.drop(['index'], axis=1)

In [12]:
events.head()

,event_id,X,Y,Z,TX,TY,chi2,signal
173027,5294,54542.996094,23498.103516,59478.0,-0.267959,0.261389,0.668103,1
483923,3009,54336.457031,49987.535156,46548.0,0.070085,0.025079,0.390442,1
794522,5586,44833.660156,57492.250000,47841.0,-0.125414,0.163239,0.360280,1
310978,29,40527.925781,28469.480469,63357.0,0.363918,-0.078953,1.989602,1
805462,5294,58759.496094,20945.890625,58185.0,-0.081031,0.061928,0.708806,1


In [13]:
import numpy as np

In [14]:
events.sort_values('event_id')

,event_id,X,Y,Z,TX,TY,chi2,signal
217938,29,40011.656250,32189.177734,69822.0,0.041732,0.439700,0.477906,1
308099,29,38609.125000,20339.082031,42669.0,0.032143,-0.047359,0.578637,1
835047,29,39567.492188,23485.277344,47841.0,0.016853,0.350911,1.303366,1
70333,29,39254.878906,28891.820312,64650.0,0.012393,0.335152,1.378288,1
558450,29,40143.570312,25234.236328,51720.0,0.211733,0.652995,0.306118,1
656683,29,38774.277344,21448.902344,42669.0,-0.524121,0.457264,2.413491,1
132371,29,39772.632812,26091.167969,62064.0,0.262677,0.176674,0.649119,1
819054,29,39619.703125,24318.421875,50427.0,0.071136,0.263867,1.535347,1
863407,29,39345.093750,25001.410156,53013.0,0.764509,-0.125135,1.503476,1
39666,29,39814.191406,22476.257812,49134.0,0.115699,0.087709,0.970971,1


In [15]:
events['track_id'] = events.groupby('event_id').cumcount()

In [16]:
events.sort_values(['event_id','track_id'])

,event_id,X,Y,Z,TX,TY,chi2,signal,track_id
310978,29,40527.925781,28469.480469,63357.0,0.363918,-0.078953,1.989602,1,0
836738,29,39118.105469,26100.332031,56892.0,-0.004092,0.308380,0.651321,1,1
262683,29,38871.402344,29117.496094,67236.0,-0.022586,0.252376,1.155281,1,2
308099,29,38609.125000,20339.082031,42669.0,0.032143,-0.047359,0.578637,1,3
187866,29,39159.531250,20733.701172,40083.0,-0.009621,0.297754,1.275150,1,4
1078706,29,39224.781250,28028.431641,62064.0,0.009552,0.334501,1.064607,1,5
1064114,29,40250.796875,33308.226562,72408.0,0.226879,0.598507,0.229727,1,6
765781,29,39117.796875,26492.003906,58185.0,0.011509,0.323582,0.870148,1,7
539582,29,40055.667969,26718.746094,55599.0,0.027660,0.878314,1.515347,1,8
423718,29,38897.789062,25557.021484,56892.0,-0.019150,0.000507,1.328074,1,9


In [17]:
# events['unique_id'] = events['event_id'].map(str) + '-' + events['track_id'].map(str)

In [18]:
events.head()

,event_id,X,Y,Z,TX,TY,chi2,signal,track_id
173027,5294,54542.996094,23498.103516,59478.0,-0.267959,0.261389,0.668103,1,0
483923,3009,54336.457031,49987.535156,46548.0,0.070085,0.025079,0.390442,1,0
794522,5586,44833.660156,57492.250000,47841.0,-0.125414,0.163239,0.360280,1,0
310978,29,40527.925781,28469.480469,63357.0,0.363918,-0.078953,1.989602,1,0
805462,5294,58759.496094,20945.890625,58185.0,-0.081031,0.061928,0.708806,1,1


In [19]:
index = pd.MultiIndex.from_arrays([events['event_id'], events['track_id']])

In [20]:
new_events = events.set_index(index)

In [21]:
new_events = new_events.sort_index()

In [22]:
new_events = new_events.drop(['event_id', 'track_id'], axis='columns')

In [23]:
new_events.head()

X             Y        Z        TX        TY  \
event_id track_id                                                            
29       0         40527.925781  28469.480469  63357.0  0.363918 -0.078953   
         1         39118.105469  26100.332031  56892.0 -0.004092  0.308380   
         2         38871.402344  29117.496094  67236.0 -0.022586  0.252376   
         3         38609.125000  20339.082031  42669.0  0.032143 -0.047359   
         4         39159.531250  20733.701172  40083.0 -0.009621  0.297754   

                       chi2  signal  
event_id track_id                    
29       0         1.989602       1  
         1         0.651321       1  
         2         1.155281       1  
         3         0.578637       1  
         4         1.275150       1

In [24]:
MAX_TRACKS=2500

In [25]:
event_idx = pd.Index(events['event_id'].unique())
track_idx = pd.RangeIndex(0, MAX_TRACKS)

In [26]:
new_index = pd.MultiIndex.from_product([event_idx, track_idx])

In [27]:
final_events = new_events.reindex(index=new_index)

In [28]:
final_events.sort_index()

X             Y        Z        TX        TY      chi2  \
29    0     40527.925781  28469.480469  63357.0  0.363918 -0.078953  1.989602   
      1     39118.105469  26100.332031  56892.0 -0.004092  0.308380  0.651321   
      2     38871.402344  29117.496094  67236.0 -0.022586  0.252376  1.155281   
      3     38609.125000  20339.082031  42669.0  0.032143 -0.047359  0.578637   
      4     39159.531250  20733.701172  40083.0 -0.009621  0.297754  1.275150   
      5     39224.781250  28028.431641  62064.0  0.009552  0.334501  1.064607   
      6     40250.796875  33308.226562  72408.0  0.226879  0.598507  0.229727   
      7     39117.796875  26492.003906  58185.0  0.011509  0.323582  0.870148   
      8     40055.667969  26718.746094  55599.0  0.027660  0.878314  1.515347   
      9     38897.789062  25557.021484  56892.0 -0.019150  0.000507  1.328074   
      10    39178.820312  20751.808594  40083.0  0.040290  0.360124  0.920168   
      11    39182.589844  25409.972656  54306.0 -0.026832  0.288388  0.638356   
      12    39282.921875  30115.078125  68529.0 -0.195433  0.335868  0.671531   
      13    38561.687500  18915.941406  65943.0  0.872508  0.805432  1.016226   
      14    40025.488281  26755.121094  59478.0  0.469917 -0.786086  2.852720   
      15    40486.285156  28227.054688  62064.0  0.000326  0.342332  1.045205   
      16    39295.398438  26037.203125  55599.0 -0.058715  0.484582  1.019731   
      17    39231.445312  28461.222656  63357.0  0.015193  0.343764  0.701367   
      18    39213.031250  24191.941406  50427.0  0.013253  0.341360  0.427439   
      19    41090.960938  32798.656250  71115.0  0.263016  0.392020  1.558477   
      20    38858.093750  29739.917969  68529.0 -0.358859  0.225263  0.654928   
      21    43329.132812  30536.070312  71115.0  0.467643 -0.504966  1.463488   
      22    41223.179688  27343.476562  54306.0  0.775893  0.493252  0.939970   
      23    40328.414062  26658.164062  56892.0 -0.098670  0.056231  1.333008   
      24    39340.597656  21849.691406  45255.0  0.116411  0.158333  0.713249   
      25    39268.906250  25790.958984  55599.0  0.117499  0.355683  1.209099   
      26    39200.019531  31317.292969  72408.0  0.061965  0.221694  0.540969   
      27    39181.304688  30107.232422  68529.0 -0.015583  0.338639  0.809449   
      28    39159.148438  19046.310547  34911.0 -0.011728  0.329553  0.807483   
      29    39198.171875  29674.539062  67236.0 -0.011988  0.342955  0.826394   
...                  ...           ...      ...       ...       ...       ...   
49513 2470           NaN           NaN      NaN       NaN       NaN       NaN   
      2471           NaN           NaN      NaN       NaN       NaN       NaN   
      2472           NaN           NaN      NaN       NaN       NaN       NaN   
      2473           NaN           NaN      NaN       NaN       NaN       NaN   
      2474           NaN           NaN      NaN       NaN       NaN       NaN   
      2475           NaN           NaN      NaN       NaN       NaN       NaN   
      2476           NaN           NaN      NaN       NaN       NaN       NaN   
      2477           NaN           NaN      NaN       NaN       NaN       NaN   
      2478           NaN           NaN      NaN       NaN       NaN       NaN   
      2479           NaN           NaN      NaN       NaN       NaN       NaN   
      2480           NaN           NaN      NaN       NaN       NaN       NaN   
      2481           NaN           NaN      NaN       NaN       NaN       NaN   
      2482           NaN           NaN      NaN       NaN       NaN       NaN   
      2483           NaN           NaN      NaN       NaN       NaN       NaN   
      2484           NaN           NaN      NaN       NaN       NaN       NaN   
      2485           NaN           NaN      NaN       NaN       NaN       NaN   
      2486           NaN           NaN      NaN       NaN       NaN       NaN   
      2487           NaN           NaN      NaN       

In [29]:
final_events['X'].shape

(250000,)

In [30]:
(final_events['X'].shape[0] - np.sum(final_events['X'].isnull()))/final_events['X'].shape[0]

0.143144

In [31]:
final_events.shape

(250000, 7)

In [32]:
fake_events = train_df[train_df['event_id'] == -999][['X', 'Y', 'Z', 'TX', 'TY', 'chi2', 'signal']].copy()

In [33]:
from functools import partial

In [34]:
def f(x):
    return np.where(np.isnan(x), 0, x)

In [35]:
final_events.apply(f, axis=1)

X             Y        Z        TX        TY      chi2  \
5294  0     54542.996094  23498.103516  59478.0 -0.267959  0.261389  0.668103   
      1     58759.496094  20945.890625  58185.0 -0.081031  0.061928  0.708806   
      2     58396.617188  20576.623047  59478.0 -0.473126 -0.210435  0.620557   
      3     58252.394531  21007.179688  59478.0 -0.087523  0.058310  0.847460   
      4     60132.343750  20679.722656  60771.0  0.116383 -0.047196  1.613834   
      5     59495.906250  20348.152344  46548.0 -0.367308  0.121312  0.730988   
      6     58277.437500  22472.255859  72408.0 -0.136258  0.123805  0.926889   
      7     58040.007812  22165.589844  71115.0 -0.103916  0.093987  0.721907   
      8     57866.343750  20706.886719  71115.0  0.326321 -0.242374  1.013186   
      9     59371.265625  20762.117188  53013.0 -0.068513  0.080501  0.353703   
      10    59057.902344  21066.675781  56892.0 -0.087905  0.038625  1.093449   
      11    60116.062500  19903.240234  41376.0 -0.063760  0.078130  0.880262   
      12    57914.976562  21999.558594  68529.0 -0.145326  0.267541  1.239583   
      13    59245.136719  21897.339844  62064.0 -0.044457  0.096512  0.600536   
      14    59691.976562  23015.414062  68529.0 -0.249684  0.647400  1.442188   
      15    58504.140625  21265.820312  62064.0 -0.074837  0.153930  0.600891   
      16    59098.484375  21808.109375  60771.0  0.145043  0.092918  0.944168   
      17    59721.261719  20396.851562  47841.0 -0.070689  0.057789  0.882577   
      18    56877.511719  24830.855469  71115.0 -0.281971  0.492825  0.781042   
      19    58521.457031  21801.667969  67236.0 -0.081390  0.084398  0.793350   
      20    60128.214844  19882.496094  41376.0 -0.060589  0.062416  1.264454   
      21    59528.101562  20676.171875  54306.0  0.235607 -0.082738  0.360168   
      22    60639.699219  21362.613281  54306.0  0.308850  0.239425  0.690721   
      23    59681.882812  20777.437500  46548.0 -0.341443  0.408464  1.909332   
      24    59686.589844  22037.613281  62064.0  0.388839  0.031073  1.636620   
      25    57772.976562  22659.726562  72408.0 -0.193797  0.020466  0.590399   
      26    58091.445312  20132.687500  50427.0 -0.314700 -0.064067  0.985019   
      27    60259.675781  21730.919922  67236.0  0.040123  0.112184  0.744838   
      28    58449.265625  21860.378906  63357.0 -0.056306  0.208045  0.760701   
      29    59789.312500  20508.472656  54306.0  0.014500 -0.031483  1.556037   
...                  ...           ...      ...       ...       ...       ...   
19167 2470      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2471      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2472      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2473      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2474      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2475      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2476      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2477      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2478      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2479      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2480      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2481      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2482      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2483      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2484      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2485      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2486      0.000000      0.000000      0.0  0.000000  0.000000  0.000000   
      2487      0.000000      0.000000      0.0  0.000

In [36]:
wrong_inputs = fake_events.sample(len(np.isnan(final_events)), replace=False).values

In [37]:
new_full_values = np.where(np.isnan(final_events.values), wrong_inputs, final_events.values)

In [38]:
new_values_df = pd.DataFrame(data=new_full_values, index=final_events.index, columns=final_events.columns)

In [39]:
new_values_df.shape

(250000, 7)

In [44]:
new_values_df.head()

X             Y        Z        TX        TY      chi2  \
5294 0  54542.996094  23498.103516  59478.0 -0.267959  0.261389  0.668103   
     1  58759.496094  20945.890625  58185.0 -0.081031  0.061928  0.708806   
     2  58396.617188  20576.623047  59478.0 -0.473126 -0.210435  0.620557   
     3  58252.394531  21007.179688  59478.0 -0.087523  0.058310  0.847460   
     4  60132.343750  20679.722656  60771.0  0.116383 -0.047196  1.613834   

        signal  
5294 0     1.0  
     1     1.0  
     2     1.0  
     3     1.0  
     4     1.0

In [40]:
# new_values_df.to_pickle('Reshape_test.pkl')

In [59]:
N_FEATURES = 7 # include target for now

In [64]:
events_array = new_values_df.values.reshape((-1, MAX_TRACKS, N_FEATURES))

In [69]:
events_array[0, :, 6]

array([1., 1., 1., ..., 0., 0., 0.])

In [70]:
# shuffling  tracks (axis 1)

In [81]:
[print(a) for a in events_array[:2, :2]]

[[ 5.45429961e+04  2.34981035e+04  5.94780000e+04 -2.67959446e-01
   2.61388570e-01  6.68103456e-01  1.00000000e+00]
 [ 5.87594961e+04  2.09458906e+04  5.81850000e+04 -8.10314342e-02
   6.19280152e-02  7.08805799e-01  1.00000000e+00]]
[[ 5.43364570e+04  4.99875352e+04  4.65480000e+04  7.00846389e-02
   2.50790548e-02  3.90441984e-01  1.00000000e+00]
 [ 5.58881172e+04  5.04235156e+04  6.46500000e+04 -1.17768787e-01
   1.68894157e-01  1.94157517e+00  1.00000000e+00]]


[None, None]

In [84]:
list(map(np.random.shuffle, events_array[:,:]))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [93]:
np.save('events_array.npy', events_array)